In [ ]:
import os
from langchain.chat_models import ChatOpenAI

# Load the OpenAI API key from a .env file first
from dotenv import load_dotenv
load_dotenv() 

chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo'
)

In [ ]:
import textwrap

def print_with_wrap(text, width=80):
    wrapped_text = textwrap.fill(text, width=width)
    print(wrapped_text)

In [ ]:
# Imports from langchain
from langchain.document_loaders import PyPDFLoader 
from langchain.embeddings import OpenAIEmbeddings 
from langchain.vectorstores import Chroma 
from langchain.chains import ConversationalRetrievalChain 
from langchain.chat_models import ChatOpenAI 

In [ ]:
# create a pdf loader
loader = PyPDFLoader("Guide-to-Programs-Services-2022.pdf")

In [ ]:
# load the pages
pages = loader.load_and_split()

In [ ]:
# Show the loaded content
for i in range(len(pages)):
    print(f'\n\n\nPage {i}')
    print(pages[i].page_content)

In [ ]:
# Get embeddings for the content and store them in a local vector database
openai_embeddings = OpenAIEmbeddings()
vectordatabase = Chroma.from_documents(pages, embedding=openai_embeddings, persist_directory=".")
vectordatabase.persist()

In [ ]:
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.9)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
crc = ConversationalRetrievalChain.from_llm(llm, vectordatabase.as_retriever(search_type="similarity", search_kwargs={"k":2}), 
            memory=memory, return_source_documents=False)

query = "Does NLHC offer scholarships?"
result = crc({"question": query})
print("Question:")
print(query)
print("Answer:")
print_with_wrap(result["answer"])

In [ ]:
query = "How can I apply for a scholarship?"
result = crc({"question": query})
print("Question:")
print(query)
print("Answer:")
print_with_wrap(result["answer"])

In [ ]:
query = "I live in Gander, what number should I call?"
result = crc({"question": query})
print("Question:")
print(query)
print("Answer:")
print(result["answer"])